## Overview

CNNPred is code that is based on the paper [CNNPred:CNN-based stock market prediction using several data sources](https://arxiv.org/pdf/1810.08923.pdf). The code implementents two versions which are CNNPred2 and CNNPred3 which use 2D and 3D input datasets.  The 2D data is time history and features for a particular time series. The 3D data is time history and features for multiple time series.  Both version produce a prediction for a particular time series which could be a market index or the price of an equity.  This is the [blog post](https://machinelearningmastery.com/using-cnn-for-financial-time-series-prediction/) that discusses the implmentation.




### Features

The following table shows the features used in CNNPred2 and CNNPred3. Note that there is a lot of economic vartiable used from FRED which we could also use. [TA-Lib](https://ta-lib.org/) is the technical indicators library they use and is one of the more popular ones boasting perhaps the largest number of features. [pandas-ta](https://github.com/twopirllc/pandas-ta) is another example and is a newer version that is designed around using pandas.


![alt text](./img/features-1of2.jpg "Features used in CNNPred 1 of 2")

![alt text](./img/features-2of2.jpg "Features used in CNNPred 2 of 2")

## Setup

In [1]:
# If you are running on a mac, make sure that you have the latest version of the OS, Xcode, and pip.
# You will probably need to install a fortran compiler for the compilation of scipy and utilities like
# cmake.  Install these using brew.

# %pip install tensorflow-macos
# %pip install scikit-learn

In [2]:
import os
import random

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, Input
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error
import warnings


In [3]:
DATADIR = "./Dataset"
TRAIN_TEST_CUTOFF = '2016-04-21'
TRAIN_VALID_RATIO = 0.75


## Evaluation Metrics

These are the evaluation metrics that are designed to be used with batches of records.

In [4]:
# https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model
# to implement F1 score for validation in a batch
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def f1macro(y_true, y_pred):
    f_pos = f1_m(y_true, y_pred)
    # negative version of the data and prediction
    f_neg = f1_m(1-y_true, 1-K.clip(y_pred,0,1))
    return (f_pos + f_neg)/2


## CNNPred2

This is the implementation for CNNPred2 which uses only one stock and associated features as a time series as input to the model.

In [5]:
def cnnpred_2d(seq_len=60, n_features=82, n_filters=(8,8,8), droprate=0.1):
    "2D-CNNpred model according to the paper"
    model = Sequential([
        Input(shape=(seq_len, n_features, 1)),
        Conv2D(n_filters[0], kernel_size=(1, n_features), activation="relu"),
        Conv2D(n_filters[1], kernel_size=(3,1), activation="relu"),
        MaxPool2D(pool_size=(2,1)),
        Conv2D(n_filters[2], kernel_size=(3,1), activation="relu"),
        MaxPool2D(pool_size=(2,1)),
        Flatten(),
        Dropout(droprate),
        Dense(1, activation="sigmoid")
    ])
    return model


In [11]:
def datagen(df: pd.DataFrame, seq_len: int, batch_size: int, targetcol: str, kind: str):
    "As a generator to produce samples for Keras model"
    batch = []
    while True:
        # Pick one dataframe from the pool
        input_cols = [c for c in df.columns if c != targetcol]
        index = df.index[df.index < TRAIN_TEST_CUTOFF]
        split = int(len(index) * TRAIN_VALID_RATIO)
        assert split > seq_len, "Training data too small for sequence length {}".format(seq_len)
        if kind == 'train':
            index = index[:split]   # range for the training set
        elif kind == 'valid':
            index = index[split:]   # range for the validation set
        else:
            raise NotImplementedError
        # Pick one position, then clip a sequence length
        while True:
            t = random.choice(index)     # pick one time step
            n = (df.index == t).argmax() # find its position in the dataframe
            if n-seq_len+1 < 0:
                continue # this sample is not enough for one sequence length
            frame = df.iloc[n-seq_len+1:n+1]
            batch.append([frame[input_cols].values, df.loc[t, targetcol]])
            break
        # if we get enough for a batch, dispatch
        if len(batch) == batch_size:
            X, y = zip(*batch)
            X, y = np.expand_dims(np.array(X), 3), np.array(y)
            yield X, y
            batch = []

def testgen(data, seq_len, targetcol):
    "Return array of all test samples"
    batch = []
    for key, df in data.items():
        input_cols = [c for c in df.columns if c != targetcol]
        # find the start of test sample
        t = df.index[df.index >= TRAIN_TEST_CUTOFF][0]
        n = (df.index == t).argmax()
        # extract sample using a sliding window
        for i in range(n+1, len(df)+1):
            frame = df.iloc[i-seq_len:i]
            batch.append([frame[input_cols].values, frame[targetcol][-1]])
    X, y = zip(*batch)
    return np.expand_dims(np.array(X),3), np.array(y)


with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)

    # Read data into pandas DataFrames.  This creates a dictionary of dataframes, one for each ticker. 
    # Each ticker is represented by a file in the datadir. 
    data = {}
    for filename in os.listdir(DATADIR):
        if not filename.lower().endswith(".csv"):
            continue # read only the CSV files
        filepath = os.path.join(DATADIR, filename)
        X = pd.read_csv(filepath, index_col="Date", parse_dates=True)
        # basic preprocessing: get the name, the classification
        # Save the target variable as a column in dataframe for easier dropna()
        name = X["Name"][0]
        del X["Name"]
        cols = X.columns
        X["Target"] = (X["Close"].pct_change().shift(-1) > 0).astype(int)
        X.dropna(inplace=True)
        # Fit the standard scaler using the training dataset
        index = X.index[X.index < TRAIN_TEST_CUTOFF]
        index = index[:int(len(index) * TRAIN_VALID_RATIO)]
        scaler = StandardScaler().fit(X.loc[index, cols])
        # Save scale transformed dataframe
        X[cols] = scaler.transform(X[cols])
        data[name] = X


In [7]:
type(data)
data.keys()

dict_keys(['NYA', 'S&P', 'NASDAQ', 'RUT', 'DJI'])

In [16]:
# data['DJI'].info()
# data['DJI'].tail()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1114 entries, 2010-10-19 to 2017-08-22
Data columns (total 83 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Close           1114 non-null   float64
 1   Volume          1114 non-null   float64
 2   mom             1114 non-null   float64
 3   mom1            1114 non-null   float64
 4   mom2            1114 non-null   float64
 5   mom3            1114 non-null   float64
 6   ROC_5           1114 non-null   float64
 7   ROC_10          1114 non-null   float64
 8   ROC_15          1114 non-null   float64
 9   ROC_20          1114 non-null   float64
 10  EMA_10          1114 non-null   float64
 11  EMA_20          1114 non-null   float64
 12  EMA_50          1114 non-null   float64
 13  EMA_200         1114 non-null   float64
 14  DTB4WK          1114 non-null   float64
 15  DTB3            1114 non-null   float64
 16  DTB6            1114 non-null   float64
 17  DGS5           

In [15]:
seq_len = 60
batch_size = 128
n_epochs = 3
n_features = 82

# Produce CNNpred as a binary classification problem
model = cnnpred_2d(seq_len, n_features)
model.compile(optimizer="adam", loss="mae", metrics=["acc", f1macro])
model.summary()  # print model structure to console

# Set up callbacks and fit the model
# We use custom validation score f1macro() and hence monitor for "val_f1macro"
checkpoint_path = "./cp2d-{epoch}-{val_f1macro:.2f}.keras"
callbacks = [
    ModelCheckpoint(checkpoint_path,
                    monitor='val_f1macro', mode="max",
                    verbose=0, save_best_only=True, save_weights_only=False, save_freq="epoch")
]
print((data))
model.fit(datagen(data, seq_len, batch_size, "Target", "train"),
          validation_data=datagen(data, seq_len, batch_size, "Target", "valid"),
          epochs=n_epochs, steps_per_epoch=400, validation_steps=10, verbose=1, callbacks=callbacks)



Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 60, 1, 8)          664       
                                                                 
 conv2d_13 (Conv2D)          (None, 58, 1, 8)          200       
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 29, 1, 8)          0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 27, 1, 8)          200       
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 13, 1, 8)          0         
 g2D)                                                            
                                                                 
 flatten_4 (Flatten)         (None, 104)              

AttributeError: 'dict' object has no attribute 'columns'

In [ ]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)

    # Prepare test data
    test_data, test_target = testgen(data, seq_len, "Target")

    # Test the model
    test_out = model.predict(test_data)
    test_pred = (test_out > 0.5).astype(int)
    print("accuracy:", accuracy_score(test_pred, test_target))
    print("MAE:", mean_absolute_error(test_pred, test_target))
    print("F1:", f1_score(test_pred, test_target))